#### ARWU

SHANGHAI RANKING网站由javascript动态加载，需要使用自动化工具（如selenium）获取多页数据。

In [ ]:
import bs4
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def get_arwu(url):
    # 初始化浏览器自动化工具
    driver = webdriver.Chrome()
    # 获取页面
    driver.get(url)
    # 设置休眠时间
    time.sleep(4)
    
    # 创建数据表存储结果
    data = pd.DataFrame(columns=['World Rank', 'Institution', 'Country/Region', 'National/Regional Rank', 'Total Score'])
    # 设置页面范围
    page_num = 1

    while True:
        # 解析页面
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # 获取存储学校数据的tr标签
        for tr in soup.find('tbody').children:
            if isinstance(tr, bs4.element.Tag):
                # 学校名称存储在标签<a>中，其余存储在各个td中
                a, tds = tr('a'), tr('td')
                # 将结果写入数据表
                data.loc[len(data.index)] = [tds[0].text.strip(), a[0].string.strip(), tds[2].div.get('style')[50:-7], tds[3].text.strip(), tds[4].text.strip()]
                # 打印结果
                print(tds[0].text.strip(), a[0].string.strip(), tds[2].div.get('style')[50:-7], tds[3].text.strip(), tds[4].text.strip())
        
        # 获取翻页按钮
        if page_num < 5:
            next_page = driver.find_element(By.XPATH,'(//a[@class="ant-pagination-item-link"])[3]') # 1-4页无“前5页”按钮
        else:
            next_page = driver.find_element(By.XPATH,'(//a[@class="ant-pagination-item-link"])[4]')

        page_num += 1
        # top200大学在前7页
        if next_page and page_num < 8:
            # 模拟翻页
            next_page.click()
            # 设置休眠时间
            time.sleep(2) 
        else:
            break
    # 将结果写入excel       
    data.to_excel('ARWU.xlsx', index=False)

In [ ]:
url = 'https://www.shanghairanking.com/rankings/arwu/2023'
get_arwu(url)